In [ ]:
#@title Step 1 : Run this cell
import sys
import shutil                                             #To play with directories and files
import os                                                 #To play with directories and files
import requests                                           #for getting Http request
from IPython.display import clear_output, display         #for clearing display
from subprocess import Popen, PIPE, getoutput             #to run bash commands
import re                                                 #regular expression for searching through html
from bs4 import BeautifulSoup                             #to get html of the page (web scrapping)
import time                                               #for delaying exec
from google.colab import files                            #to play with gdrive files
from IPython.core.display import display, HTML



# Creating Torrent Class
class Torrent:

  def __init__(self):
    self.Title_list = []          #variable for Title list
    self.magnet_list = []         #variable for Magnet list
    self.detail_list = []         #variable for detail list (size, seeders, leechers)
    self.detail_append = []
    self.date_list = []           #variable for upload date list
    self.download_list = []       #variable for download list
    self.page = 1                 #Variable to indicate current page
    self.search_term=''           #Variable for search term
    self.down_mem = 0             #Variable for total torrent memory


  #function to enter serach term
  def get_search(self):
    self.page = 1
    self.search_term = input('Enter Search Term  :  ')
    self.Search_list()


  #function to get search list
  def Search_list(self):
    self.Title_list = []
    self.magnet_list = []
    self.detail_list = []
    detail_append = []
    self.date_list = []
    det=0                     #temporary variable to get torrent details

    #Piratebay.party is the root website that is being scrapped with page number
    search_url = 'https://piratebay.party/search/' + self.search_term + '/'+str(self.page)+'/99/0'

    #getting search results from piratebay.party
    search_page = requests.get(search_url)

    #converting to html
    search_soup = BeautifulSoup(search_page.text, "lxml")

    #scrapping through HTML
    for link in search_soup.find_all("td"):

            #Getting titles of results
            if 'title="Details for'  in str(link):
                self.Title_list.append(re.search(r'title="Details for (.*?)">', str(link)).group(1))

            #Getting magnet links of results
            if 'magnet:?'  in str(link):
                self.magnet_list.append(re.search(r'<a href="(.*?)" title', str(link)).group(1))

            #Getting torrent details
            if 'align="right"'  in str(link):
                detail_append.append(re.search(r'"right">(.*?)</td>', str(link)).group(1))
                det+=1
            if (det==3):
                self.detail_list.append(detail_append)
                det=0
                detail_append=[]

            #Getting torrent upload dates
            if re.findall(r'\d\d-\d\d\s\d{4}', str(link)) or re.findall(r'\d\d:\d\d',str(link)):
                self.date_list.append(re.search(r'<td>(.*?)</td>', str(link)).group(1))

    #displaying the results
    if (self.Title_list==[]):

        #message if no results found
        print(f"Sorry! No Search Results")

    else:

        #printing if results found
        html_str = '<table><tr><td style="width:10%"><b>Sl. No</b></td><td style="width:40%"><b>TITLE</b></td><td style="width:10%"><b>Upload Date</b></td><td style="width:10%"><b>Size</b></td><td style="width:10%"><b>Seeders</b></td><td style="width:10%"><b>Leachers</b></td><td style="width:10%"><b>Magnet</b></td></tr>'

        for index in range(0,len(self.Title_list)):
            k = ((self.page-1)*30) + index+1
            try:
               html_str+=f"<tr><td>{k}</td><td><a href={self.magnet_list[index]}>{self.Title_list[index]}></a></td><td>{self.date_list[index]}</td><td> {self.detail_list[index][0]}</td> <td>{self.detail_list[index][1]}</td> <td>{self.detail_list[index][2]}</td><td> <a href={self.magnet_list[index]}>Magnet</a> </td></tr>"
            except:
               pass
        html_str+="</table>"
        display(HTML(html_str))
    return


In [ ]:
#@title Step 2: Search
if __name__=="__main__":


#Create a torrent class
  Test1 = Torrent()
  Test1.get_search()

  key=''
  while (True):
    #start searching torrent
    if (Test1.Title_list == []):
      Test1.get_search()
      continue

    #get input from user
    else:
      time.sleep(0.5)
      key = input("'N': Next page, 'P': Previous page, 'S': change search term   :- ").strip()
      #print(key)
      if (key.lower()=='n' or key.lower()=='p' or key.lower()=='s'):
        pass
      else:
        print("Enter Valid input!")
        continue


    #Display next page
    if (key.lower()=='n'):
      clear_output(wait=True)
      time.sleep(1)
      Test1.page += 1
      Test1.Search_list()

    #Display previous page
    if (key.lower()=='p'):
      clear_output(wait=True)
      time.sleep(1)
      if (Test1.page==1):
        print("Previous Page Not Available!")
        continue
      Test1.page -= 1
      Test1.Search_list()

    #Search another term
    if (key.lower()=='s'):
      clear_output(wait=True)
      time.sleep(1)
      Test1.get_search()
